# Instructions

This notebooks processes the metric provided by CUDA Flux into the features used by GPU mangrove. You don't need to run this notebook as the database containing the processed features already comes with this project.

The notebook can be run executiong all cells at once or step by step to explore the data inbetween steps. 

# Setup

In [1]:
# Setup Imports
import sqlite3
import numpy as np
import pandas as pd
import re

import time
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression

from IPython.core.display import display, HTML

In [2]:
# Notebook Layout - Run to set the width of jupyter cells to 100%
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

# DB Paths, excluded apps

In [4]:
# Input
CUDAFluxMetricDB = 'data/CUDAFlux-0.3-Metrics.db'
# Output
FeatureTableDB = 'data/FeatureTable-0.3.db'

# List of Applications that should be excluded from the final db
excluded_apps = [('rodinia-3.1','srad-v1'),
                ('rodinia-3.1','srad-v2')]

# Read Features

In [5]:
# Get all Column and Row Labels
with sqlite3.Connection(CUDAFluxMetricDB) as conn:
    cur = conn.cursor()

    cur.execute('select distinct metric from fluxmetrics')
    metrics = cur.fetchall()
    cur.execute('select distinct bench,app,dataset,lseq from fluxmetrics')
    kernels = cur.fetchall()

    print(len(metrics), "Columns,", len(kernels), "Rows")
    
# Consistent Mapping/Order of rows and columns
row = {}
count = 0
for kernel in kernels:
    row[kernel] = count
    count += 1
col = {}
count = 0
for metric in metrics:
    col[metric[0]] = count
    count += 1

236 Columns, 965776 Rows


In [6]:
# Build ndarray with all Features
with sqlite3.Connection(CUDAFluxMetricDB) as conn:
    data = np.zeros((len(kernels),len(metrics)))

    cur = conn.cursor()
    cur.execute('select * from fluxmetrics')
    print("Processing Items..")
    for item in log_progress(cur.fetchall()):
        data[row[item[0:4]], col[item[5]]] = item[-1]
        
    # Build Dataframe for Labeled Storage
    df_f = pd.DataFrame(data, 
                  index=pd.Index(kernels, names=['bench', 'app', 'dataset', 'lseq']), 
                  columns=[item[0] for item in metrics])

Processing Items..


In [7]:
# Check for Errors
if False:
    with sqlite3.Connection(CUDAFluxMetricDB) as conn:
        cur = conn.cursor()
        cur.execute("select * from fluxmetrics where bench=? and app=? and dataset=? and lseq=?", kernels[0])
        res = cur.fetchall()
        print(data[0,:])
        print([x[-1] for x in res])

# Process Features

## Drop Faulty Records

In [8]:
for app in excluded_apps:
    df_f.drop(app, inplace=True)

/share/lbraun/opt/anaconda3/lib/python3.7/site-packages/IPython/core/async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)
/share/lbraun/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


## Features Engineering

### Helper Functions

In [9]:
# Define how instructions are grouped
inst_groups = {# Integer Inst
               r'(add|sub|mul|mad|mul24|mad24|sad|div|rem|abs|neg|min|max|popc|clz|bfind|fns|brev|bfe|bfi|dp4a|dp2a)[\w.]*?.[us](\d+)$' : 'int',
               # ToDo: extended precission int inst (not found yet)
               # Floating Inst
               r'(testp|copysign|add|sub|mul|fma|mad|div|abs|neg|min|max|rcp)[\w.]*?.f(\d+)$' : 'float',
               r'(sqrt|rsqrt|sin|cos|lg2|ex2)[\w.]*?.f(\d+)$' : 'float.special',
               # Comparing and Selection inst
               r'(set|setp|selp|slct)[\w.]*?.\w(\d+)$' : 'comp_sel',
               # Logic Inst
               r'(and|or|xor|not|cnot|lop3|shf|shl|shr)[\w.]*?.\w?(\d+|pred)$' : 'logic',
               # Data Movement Inst
               r'(ld.global)(.v[42])?.\w(\d+)$' : 'ld.global',
               r'(ld)(.v[42])?.\w(\d+)$' : 'ld.global',
               r'(ld.shared)(.v[42])?.\w(\d+)$' : 'ld.shared',
               r'(ld.volatile.shared)(.v[42])?.\w(\d+)$' : 'ld.shared',
               r'(ld.const)(.v[42])?.\w(\d+)$' : 'ld.const',
               r'(ld.local)(.v[42])?.\w(\d+)$' : 'ld.local',
               r'(ld.param)(.v[42])?.\w(\d+)$' : 'ld.param',
               r'(st.global)(.v[42])?.\w(\d+)$' : 'st.global',
               r'(st)(.v[42])?.\w(\d+)$' : 'st.global',
               r'(st.shared)(.v[42])?.\w(\d+)$' : 'st.shared',
               r'(st.volatile.shared)(.v[42])?.\w(\d+)$' : 'st.shared',
               r'(st.const)(.v[42])?.\w(\d+)$' : 'st.const',
               r'(st.local)(.v[42])?.\w(\d+)$' : 'st.local',
               r'(st.param)(.v[42])?.\w(\d+)$' : 'st.param',
               r'(mov)[\w.]*?.\w?(\d+|pred)$' : 'mov',
               # Data Conversion
               r'(cvta|cvt)[\w.]*?.\w(\d+)$' : 'cvt',
               # Control Flow
               r'(bra|call|ret|exit)[\w.]*?$' : 'control',
               # Atomic Inst
               r'(atom.global)[\w.]*?(\d+)$' : 'atom.global',
               r'(atom.shared)[\w.]*?(\d+)$' : 'atom.shared',
               # Sync
               r'bar.sync' : 'bar.sync'
               # End
              }

In [10]:
# Helper function to actually group instructions
def meltInstructions(insts, inst_groups = inst_groups):
    inst_map = { }

    for inst in insts:
        if inst in { 'gX', 'gY', 'gZ', 'bX', 'bY', 'bZ', 'shm', 'time'}:
            inst_map[inst] = [inst]
            continue
        m = None
        for ex in inst_groups:
            m = re.match(ex, inst)
            if( m is not None):
                num_groups = len(m.groups())
                #print(inst, m.groups())
                if (num_groups >= 3):
                    if m.group(num_groups-1) is None:
                        key = inst_groups[ex]+'.'+m.group(num_groups)
                    else:
                        key = inst_groups[ex]+'.'+str(int(m.group(num_groups-1)[2:])*int(m.group(num_groups)))
                elif (num_groups >= 2):
                    if m.group(num_groups) == 'pred':
                        key = inst_groups[ex]+'.'+'32'
                    else:
                        key = inst_groups[ex]+'.'+m.group(num_groups)
                else:
                    key = inst_groups[ex]
                #print(inst, inst_groups[ex], key, m.groups())
                if key in inst_map:
                    inst_map[key].append(inst)
                else:
                    inst_map[key] = [inst]
                break
        if (m == None):
            if 'misc' in inst_map:
                inst_map['misc'].append(inst)
            else:
                inst_map['misc'] = [inst]
            print(inst, '\033[1;31mNo Match!\033[0m')
    return inst_map

### Group Features

In [11]:
# Produce map for summing up columns
inst_map = meltInstructions(df_f.columns.values)

# View if groups are correct
inst_map

clz.b32 No Match!


{'gX': ['gX'],
 'gY': ['gY'],
 'gZ': ['gZ'],
 'bX': ['bX'],
 'bY': ['bY'],
 'bZ': ['bZ'],
 'shm': ['shm'],
 'time': ['time'],
 'ld.param.64': ['ld.param.u64', 'ld.param.f64'],
 'ld.param.32': ['ld.param.u32', 'ld.param.f32'],
 'cvt.32': ['cvt.s64.s32',
  'cvt.u64.u32',
  'cvt.rn.f64.s32',
  'cvt.f64.f32',
  'cvt.u16.u32',
  'cvt.rn.f32.s32',
  'cvt.rmi.f32.f32',
  'cvt.rzi.s32.f32',
  'cvt.rn.f32.u32',
  'cvt.rzi.u32.f32',
  'cvt.rzi.f32.f32',
  'cvt.rni.s32.f32'],
 'int.64': ['add.s64', 'mul.lo.s64', 'sub.s64', 'neg.s64', 'add.u64'],
 'comp_sel.64': ['setp.le.u64',
  'setp.ge.u64',
  'setp.lt.u64',
  'setp.ne.s64',
  'setp.eq.s64',
  'setp.neu.f64',
  'setp.eq.f64',
  'selp.f64',
  'setp.lt.f64',
  'setp.ge.f64',
  'setp.lt.s64',
  'setp.gt.s64',
  'setp.leu.f64',
  'setp.gtu.f64',
  'setp.gt.u64',
  'setp.ge.s64'],
 'control': ['bra', 'ret', 'bra.uni', 'call.uni'],
 'mov.32': ['mov.u32', 'mov.b32', 'mov.f32', 'mov.pred'],
 'int.32': ['mad.lo.s32',
  'add.s32',
  'mul.wide.u32',
  'mu

In [12]:
# Create grouped dataframe
dfg = pd.DataFrame(index=df_f.index, columns=[key for key in inst_map])
for key in log_progress(inst_map):
    #print("Processing", key)
    dfg[key] = (df_f[inst_map[key]].sum(axis=1))

### Apply Analytical Features

In [13]:
# Analytical Metrics

feature_map = {}

for col in [ 'gX', 'gY', 'gZ', 'bX', 'bY', 'bZ', 'shm']:
    feature_map[col] = dfg[col].values

feature_map['CTAs'] = dfg['gX'].values * dfg['gZ'].values * dfg['gZ'].values
feature_map['BlockThreads'] = dfg['bX'].values * dfg['bZ'].values * dfg['bZ'].values
feature_map['TotalThreads'] = feature_map['CTAs'] * feature_map['BlockThreads']

def compute_data_vol(df, prefix, totalThreads):
    cols = set()
    for col in df.columns.values:
        if col.startswith(prefix):
            cols.add(col)
    res = None
    for col in cols:
        if res is None:
            res = df[col] * int(col.split('.')[-1])/8
        else: 
            res += df[col] * int(col.split('.')[-1])/8
    return res * totalThreads

feature_map['V_ldGlobal'] = compute_data_vol(dfg, 'ld.global.', feature_map['TotalThreads'])
feature_map['V_stGlobal'] = compute_data_vol(dfg, 'st.global.', feature_map['TotalThreads'])
feature_map['V_ldShm'] = compute_data_vol(dfg, 'ld.shared.', feature_map['TotalThreads'])
feature_map['V_stShm'] = compute_data_vol(dfg, 'st.shared.', feature_map['TotalThreads'])
feature_map['V_ldParam'] = compute_data_vol(dfg, 'ld.param.', feature_map['TotalThreads'])
feature_map['V_stParam'] = compute_data_vol(dfg, 'st.param.', feature_map['TotalThreads'])
feature_map['V_ldLocal'] = compute_data_vol(dfg, 'ld.local.', feature_map['TotalThreads'])
feature_map['V_stLocal'] = compute_data_vol(dfg, 'st.local.', feature_map['TotalThreads'])
feature_map['V_ldConst'] = compute_data_vol(dfg, 'ld.const.', feature_map['TotalThreads'])

cols_inst = ['float.32', 'logic.64', 'comp_sel.32', 'int.32', 'cvt.32',
       'comp_sel.64', 'mov.32', 'logic.32', 'control',
       'int.64', 'cvt.64', 'ld.global.32', 'ld.param.64',
       'st.global.32', 'ld.shared.32', 'mov.64',
       'st.shared.32', 'ld.param.32', 'bar.sync', 'float.64',
       'ld.const.64', 'ld.const.32', 'misc', 'ld.local.32', 'st.local.32',
       'st.global.64', 'atom.shared.32', 'ld.shared.64', 'st.shared.64',
       'comp_sel.16', 'st.global.8', 'ld.global.8', 'mov.16',
       'ld.global.64', 'int.16', 'ld.local.8', 'st.local.8',
       'st.param.64', 'st.param.32', 'atom.global.32', 'logic.16',
       'ld.global.128', 'ld.local.64', 'st.local.64', 'cvt.16',
       'st.global.128', 'ld.shared.8', 'ld.param.8','float.special.32','float.special.64']

feature_map['total_inst'] = dfg[cols_inst].values.sum(axis=1) * feature_map['TotalThreads']
for col in cols_inst:
    a = dfg[col].values * feature_map['TotalThreads']
    b = feature_map['total_inst']
    feature_map['P_' + col] = np.divide(a, b, out=np.zeros_like(a), where=b!=0)
    
for col in cols_inst:
    feature_map[col] = dfg[col].values * feature_map['TotalThreads'] 

features = pd.DataFrame(feature_map)

In [14]:
# Analytical Metrics

feature_map = {}

for col in [ 'gX', 'gY', 'gZ', 'bX', 'bY', 'bZ', 'shm']:
    feature_map[col] = dfg[col].values

feature_map['CTAs'] = dfg['gX'].values * dfg['gY'].values * dfg['gZ'].values
feature_map['BlockThreads'] = dfg['bX'].values * dfg['bY'].values * dfg['bZ'].values
feature_map['TotalThreads'] = feature_map['CTAs'] * feature_map['BlockThreads']

def compute_data_vol(df, prefix):
    cols = set()
    for col in df.columns.values:
        if col.startswith(prefix):
            cols.add(col)
    res = None
    for col in cols:
        if res is None:
            res = df[col] * int(col.split('.')[-1])/8
        else: 
            res += df[col] * int(col.split('.')[-1])/8
    return res

feature_map['V_ldGlobal'] = compute_data_vol(dfg, 'ld.global.')
feature_map['V_stGlobal'] = compute_data_vol(dfg, 'st.global.')
feature_map['V_ldShm'] = compute_data_vol(dfg, 'ld.shared.')
feature_map['V_stShm'] = compute_data_vol(dfg, 'st.shared.')
feature_map['V_ldParam'] = compute_data_vol(dfg, 'ld.param.')
feature_map['V_stParam'] = compute_data_vol(dfg, 'st.param.')
feature_map['V_ldLocal'] = compute_data_vol(dfg, 'ld.local.')
feature_map['V_stLocal'] = compute_data_vol(dfg, 'st.local.')
feature_map['V_ldConst'] = compute_data_vol(dfg, 'ld.const.')

cols_inst = ['float.32', 'logic.64', 'comp_sel.32', 'int.32', 'cvt.32',
       'comp_sel.64', 'mov.32', 'logic.32', 'control',
       'int.64', 'cvt.64', 'ld.global.32', 'ld.param.64',
       'st.global.32', 'ld.shared.32', 'mov.64',
       'st.shared.32', 'ld.param.32', 'bar.sync', 'float.64',
       'ld.const.64', 'ld.const.32', 'misc', 'ld.local.32', 'st.local.32',
       'st.global.64', 'atom.shared.32', 'ld.shared.64', 'st.shared.64',
       'comp_sel.16', 'st.global.8', 'ld.global.8', 'mov.16',
       'ld.global.64', 'int.16', 'ld.local.8', 'st.local.8',
       'st.param.64', 'st.param.32', 'atom.global.32', 'logic.16',
       'ld.global.128', 'ld.local.64', 'st.local.64', 'cvt.16',
       'st.global.128', 'ld.shared.8', 'ld.param.8','float.special.32','float.special.64']

feature_map['total_inst'] = dfg[cols_inst].values.sum(axis=1)
# Compute percentual values for the features
#for col in cols_inst:
#    a = dfg[col].values
#    b = feature_map['total_inst']
#    feature_map['P_' + col] = np.divide(a, b, out=np.zeros_like(a), where=b!=0)
    
for col in cols_inst:
    feature_map[col] = dfg[col].values

features = pd.DataFrame(feature_map)

### Dump Table into DB

In [15]:
# Add Column with Kernel Names
with sqlite3.Connection(CUDAFluxMetricDB) as conn:
    # Build Pandas Dataframe from SQL Query
    df_name = pd.read_sql_query(
        'select distinct bench,app,dataset,lseq,name from fluxmetrics', 
        conn, index_col=['bench','app','dataset','lseq'])
    for app in excluded_apps:
        df_name.drop(app, inplace=True)

In [16]:
if features.shape[0] == df_name.shape[0]:
    features_name = features.join(df_name)
else:
    print("Shape mismatch!")

In [17]:
# View Dataframe
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(features_name[features_name.index.get_level_values(1) == '2mm.exe'])

gX      gY   gZ    bX   bY   bZ  \
bench             app     dataset lseq                                       
polybench-gpu-1.0 2mm.exe s2      0     128.0   512.0  1.0  32.0  8.0  1.0   
                                  1     128.0   512.0  1.0  32.0  8.0  1.0   
                          s3      0     256.0  1024.0  1.0  32.0  8.0  1.0   
                                  1     256.0  1024.0  1.0  32.0  8.0  1.0   
                          s1      0      64.0   256.0  1.0  32.0  8.0  1.0   
                                  1      64.0   256.0  1.0  32.0  8.0  1.0   

                                        shm      CTAs  BlockThreads  \
bench             app     dataset lseq                                
polybench-gpu-1.0 2mm.exe s2      0     0.0   65536.0         256.0   
                                  1     0.0   65536.0         256.0   
                          s3      0     0.0  262144.0         256.0   
                                  1     0.0  262144.0         256.0   
                          s1      0     0.0   16384.0         256.0   
                                  1     0.0   16384.0         256.0   

                                        TotalThreads    V_ldGlobal  \
bench             app     dataset lseq                               
polybench-gpu-1.0 2mm.exe s2      0       16777216.0  5.498229e+11   
                                  1       16777216.0  5.498229e+11   
                          s3      0       67108864.0  4.398315e+12   
                                  1       67108864.0  4.398315e+12   
                          s1      0        4194304.0  6.873625e+10   
                                  1        4194304.0  6.873625e+10   

                                          V_stGlobal  V_ldShm  V_stShm  \
bench             app     dataset lseq                                   
polybench-gpu-1.0 2mm.exe s2      0     2.748779e+11      0.0      0.0   
                                  1     2.748779e+11      0.0      0.0   
                          s3      0     2.199023e+12      0.0      0.0   
                                  1     2.199023e+12      0.0      0.0   
                          s1      0     3.435974e+10      0.0      0.0   
                                  1     3.435974e+10      0.0      0.0   

                                           V_ldParam  V_stParam  V_ldLocal  \
bench             app     dataset lseq                                       
polybench-gpu-1.0 2mm.exe s2      0     5.368709e+08        0.0        0.0   
                                  1     5.368709e+08        0.0        0.0   
                          s3      0     2.147484e+09        0.0        0.0   
                                  1     2.147484e+09        0.0        0.0   
                          s1      0     1.342177e+08        0.0        0.0   
                                  1     1.342177e+08        0.0        0.0   

                                        V_stLocal  V_ldConst    total_inst  \
bench             app     dataset lseq                                       
polybench-gpu-1.0 2mm.exe s2      0           0.0        0.0  4.131222e+11   
                                  1           0.0        0.0  4.131222e+11   
                          s3      0           0.0        0.0  3.301756e+12   
                                  1           0.0        0.0  3.301756e+12   
                          s1      0           0.0        0.0  5.174093e+10   
                                  1           0.0        0.0  5.174093e+10   

                                            float.32     logic.64  \
bench             app     dataset lseq                              
polybench-gpu-1.0 2mm.exe s2      0     6.871948e+10  117440512.0   
                                  1     6.871948e+10  117440512.0   
                          s3      0     5.497558e+11  469762048.0   
                                  1     5.497558e+11  469762048.0   
                          s1   

In [18]:
# Dump Dataframe into SQL Table
with sqlite3.connect(FeatureTableDB) as conn:
    features_name.to_sql('fluxfeatures',conn)
    conn.commit()